In [1]:
# required imports

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import requests
import urllib3
from bs4 import BeautifulSoup

import selenium
import time
import re

In [2]:
def get_url(uni, year):
    url = (
        f'''https://{uni}.rivals.com/search#?formValues='''
        '''{'''
        '''"sport":"Football",'''
        f'''"recruit_year":{year},'''
        f'''"prospect_profiles.prospect_colleges.college_common_name":["{uni}"],'''
        '''"offer_and_visit_type":["Offer"],'''
        '''"prospect_profiles.prospect_colleges.offer":true,'''
        '''"page_number":1,"page_size":100'''
        '''}'''
        )
    return url

In [3]:
get_url("Auburn", 2013)

'https://Auburn.rivals.com/search#?formValues={"sport":"Football","recruit_year":2013,"prospect_profiles.prospect_colleges.college_common_name":["Auburn"],"offer_and_visit_type":["Offer"],"prospect_profiles.prospect_colleges.offer":true,"page_number":1,"page_size":100}'

In [4]:
# launch browser

browser = webdriver.Firefox()

In [5]:
uni = "Auburn"
year = 2014

url = get_url(uni, year)

In [6]:
browser.get(url)

In [7]:
search_button_xpath = """
//*[@id="articles"]/div/div[2]/div/div/div[1]/form/div[2]/div[5]/button
"""

In [8]:

search_button = browser.find_element_by_xpath(search_button_xpath)

In [9]:
search_button.click()

In [10]:
soup = BeautifulSoup(browser.page_source)

In [11]:
table = soup.select_one("table.people")

In [12]:
table.text

'\n\n\n\n\nNAME\n\n\n\nYEAR\n\n\n\nPOS\n\n\n\nSTARS\n\n\n\nRR\n\n\n\nHT\n\n\n\nWT\n\n\n\nHOMETOWN\n\n\n\nRANK\n\n\n\nSCHOOL\n\n\n\n\n\n\n\n\n            Show unofficial visits\n          \n\n\n\n\n\n'

In [13]:
rows = table.select('tr')

In [14]:
string.ascii_letters, string.ascii_lowercase, string.ascii_uppercase, string.digits, string.punctuation, string.whitespace

NameError: name 'string' is not defined

In [ ]:
import string
whitespace = set(string.whitespace)

def clean_text(text):
    """Replace all sequences of whitespace in text with a single space."""
    output = []
    in_whitespace = True
    for char in text:
        if char in whitespace:
            if not in_whitespace:
                output.append(" ")
                in_whitespace = True
        else:
            in_whitespace = False
            output.append(char)
    return ''.join(output).strip()
            

In [ ]:
data = [[clean_text(td.text) for td in row.select('td')]
        for row in rows]

In [ ]:
headers = [clean_text(th.text) for th in table.select('th')]

In [ ]:
headers

In [ ]:
data

In [ ]:
df = pd.DataFrame(data, columns=headers)[1:]

In [ ]:
df.head()

In [ ]:
df['OFFER'] = uni

In [ ]:
df

In [ ]:
import pymongo
mc = pymongo.MongoClient()
db = mc['rivals']
offers_coll = db['offers']

In [ ]:
# Convert DF into format that Mongo likes (list of dict)
offers_data = df.to_dict(orient="records")
offers_data[:2]

In [ ]:
# Save this batch to MongoDB
offers_coll.insert_many(offers_data)

In [ ]:
# Retrieve all from MongoDB
retrieved_data = list(offers_coll.find())
pd.DataFrame(retrieved_data)